In [2]:
import numpy as np
import json
import pandas as pd
import recordlinkage
import re

In [24]:
vivino = pd.read_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv', encoding='utf-8')

direktimport = pd.read_csv('/home/p/wines/winerecommender/data-matching/direktimport_formatted.csv', encoding='utf-8')
divino = pd.read_csv('/home/p/wines/winerecommender/data-matching/divino_formatted.csv', encoding='utf-8')
#jacques = pd.read_csv('home/p/wines/winerecommender/data-matching/jacques_wines.csv', encoding='utf-8')

In [25]:
vivino["wine_name"] = vivino["wine_name"].str.lower()
vivino["wine_winery"] = vivino["wine_winery"].str.lower()
direktimport["wine_name"] = direktimport["wine_name"].str.lower()
direktimport["wine_winery"] = direktimport["wine_winery"].str.lower()


In [51]:
indexer = recordlinkage.Index()
indexer.block(left_on='wine_type', right_on='wine_type')
indexer.block(left_on='wine_country', right_on='wine_country')

candidate_pairs = indexer.index(vivino, direktimport)

In [52]:
compare_wines = recordlinkage.Compare()
compare_wines.string('wine_name', 'wine_name', method='jarowinkler', threshold=0.8, label='name')
compare_wines.string('wine_winery', 'wine_winery', method='jarowinkler', threshold=0.80, label='winery')
compare_wines.numeric('wine_alcohol', 'wine_alcohol', label = 'alc')
compare_wines.string('wine_region', 'wine_region', method='jarowinkler', threshold=0.80, label='region')

<Compare>

In [53]:
X = compare_wines.compute(candidate_pairs, vivino, direktimport)
res = X[X.sum(axis=1) > 3]
res.loc[0]

In [76]:
import datasketch
import fuzzymatcher

ons = ['wine_name', 'wine_winery', 'wine_country', 'wine_region', 'wine_type']
vivinof = vivino[ons]
direktimportf = direktimport[ons]
resf = fuzzymatcher.fuzzy_left_join(direktimportf, vivinof, ons, ons)

In [77]:
resf.sort_values('best_match_score', ascending = False, inplace=True)

In [80]:
resf.to_csv('/home/p/wines/winerecommender/data-matching/fuzzy_results.csv')




In [104]:
resf['search'] = resf['wine_winery_left'] +' '+ resf['wine_name_left']
resf['index'] = list(range(len(resf['search'])))
resf.set_index('index', inplace=True)
strings = list(resf['search'][170:])
ids = list(resf['__id_left'].map(lambda x: x.split('_')[0])[170:])
i=0
out = []
for string in strings:
    out.append({
        "local_id": ids[i],
        "search_string": string
    })
    i+=1


In [106]:
with open("/home/p/wines/winerecommender/data-matching/search.json", "w") as f:
    f.write(json.dumps(out))

# grab test data for recommendations

In [ ]:
with open('/home/p/wines/winerecommender/WineRecommender_API/recommender_engine/example_json/example_input_profile.json') as f:
    profile = json.load(f)

with open('/home/p/wines/winerecommender/WineRecommender_API/recommender_engine/example_json/example_input_wines.json') as f:
    wines = json.load(f)

In [11]:
def calcRecs(wines_list, user_profile, num_wines, structure_param = 0.5, tastes_param = 1, ratings_param = 1):
    user_profile["structure"] = np.array(user_profile["structure"].values())
    user_profile["tastes"] = np.array(user_profile["tastes"].values())
    for wine in wines_list:
        wine["wine_tastes"] = np.array(wine["wine_tastes"].values())
        wine["wine_structure"] = np.array(wine["wine_structure"].values())
        wine["wine_rating"] = wine["wine_rating"]/4
        wine["score"] = (structure_param * sum(user_profile["structure"]*wine["wine_structure"]) + tastes_param * sum(user_profile["tastes"]*wine["wine_tastes"])) * (ratings_param * wine["wine_rating"])
    
    wines_list = sorted(wines_list, key=lambda k: k["score"])
    return wines_list[:num_wines-1]

In [3]:
import json
with open('/home/p/wines/winerecommender/data-matching/vivino_data_combined_fm.json') as f:
    viv = json.load(f)

In [7]:
seen = set()
keys = []
for wine in viv:
    try:
        for group in wine["wine_tastes"]:
            if group["group"] not in seen:
                seen.add(group["group"])
                keys.append(group["group"])
    except:
        continue
print(sorted(keys))

['black_fruit', 'citrus_fruit', 'dried_fruit', 'earth', 'floral', 'microbio', 'non_oak', 'oak', 'red_fruit', 'spices', 'tree_fruit', 'tropical_fruit', 'vegetal']
